In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import warnings

# Dependency imports
from absl import flags
import matplotlib as plt
import numpy as np
import pickle
from PIL import Image

import tensorflow as tf
import tensorflow_probability as tfp

import keras



import bayesian_cnn
import config as cf
import util

warnings.filterwarnings("ignore")

Using TensorFlow backend.
C:\Users\ammmy\Miniconda3\envs\tf\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\ammmy\Miniconda3\envs\tf\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\ammmy\Miniconda3\envs\tf\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\ammmy\Miniconda3\envs\tf\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\ammmy\Miniconda3\envs\tf\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f

In [2]:
root_dir = 'Data'
train_dir = os.path.join('Data','Train')
test_dir = os.path.join('Data','Test')

#util.to3_channel(train_dir)
#util.to3_channel(test_dir)
if not os.path.isfile(os.path.join(root_dir,"Preprocessed_Data", "preprocessed_train.p")):
        print ("Processed File doesn't exist. Preprocessing data first")
        train_data = util.preprocess_and_save_data(data_type='train')
else:
    print ("Loading the training data")
    train_data= pickle.load(open(os.path.join(root_dir,"Preprocessed_Data","preprocessed_train.p"),"rb"))

X_train, y_train = train_data['features'], train_data['labels']


if not os.path.isfile(os.path.join(root_dir, "Preprocessed_Data", "preprocessed_test.p")):
    print ("Processed File doesn't exist. Preprocessing data first")
    test_data = util.preprocess_and_save_data(data_type='test')
else:
    print ("Loading the testing data")
    test_data= pickle.load(open(os.path.join(root_dir,"Preprocessed_Data","preprocessed_test.p"),"rb"))

X_test, y_test = test_data['features'], test_data['labels']


Loading the training data
Loading the testing data


In [3]:
print("Shape of X_train is ", X_train.shape)
print("Shape of X_test is ", X_test.shape)
print("Shape of y_train is ", y_train.shape)
print("Shape of y_test is ", y_test.shape)
(images, targets, iter_handle, train_iterator, test_iterator) = util.build_data_pipeline(X_train, X_test,y_train, y_test)

model, logits, targets_distribution = bayesian_cnn.bayesian_alexnet(images)

Shape of X_train is  (2789, 227, 227, 3)
Shape of X_test is  (693, 227, 227, 3)
Shape of y_train is  (2789,)
Shape of y_test is  (693,)


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16846040962972347550
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4952306483
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12642457603718843845
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [ ]:
# Compute the -ELBO as the loss, averaged over the batch size.
neg_log_likelihood = -tf.reduce_mean(targets_distribution.log_prob(targets))
kl = sum(model.losses) / X_train.shape[0]
elbo_loss = neg_log_likelihood + kl

# Defining metrics for evalution
predictions = tf.argmax(logits, axis=1)
accuracy, accuracy_update_op = tf.metrics.accuracy(labels=targets, predictions = predictions)

with tf.name_scope("train"):
    optimizer =  tf.train.AdamOptimizer(learning_rate=cf.learning_rate)
    #optimizer = tf.keras.optimizers.SGD(lr = cf.learning_rate, momentum = cf.momentum, decay = cf.decay)
    train_op = optimizer.minimize(elbo_loss)

# Extract weight posterior statistics for layers with weight distributions
# for later visualization.
names = []
qmeans = []
qstds = []

for i, layer in enumerate(model.layers):
    try:
        q = layer.kernel_posterior
    except AttributeError:
        continue
    names.append("Layer {}".format(i))
    qmeans.append(q.mean())
    qstds.append(q.stddev())

# Initialize the variables
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

with tf.Session() as sess:
    sess.run(init_op)
    print("Running the training loop")
    # Run the training loop.
    train_handle = sess.run(train_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())
    for step in range(cf.epochs):
        _ = sess.run([train_op, accuracy_update_op], feed_dict={iter_handle: train_handle})
        
        if step % 2== 0:
            loss_value, accuracy_value = sess.run([elbo_loss, accuracy], feed_dict={iter_handle: train_handle})
            
            print("Epoch: {:>3d} Loss: {:.3f} Accuracy: {:.3f}".format(step, loss_value, accuracy_value))

    #Sampling from the posterior and obtaining mean probability for held out dataset
    probs = np.asarray([sess.run((targets_distribution.probs), feed_dict={iter_handle: test_handle})
                        for _ in range(cf.num_monte_carlo)])
    mean_probs = np.mean(probs, axis=0)

    test_acc_dist = []
    for prob in probs:
        y_test_pred = np.argmax(prob, axis = 1).astype(np.float32)
        accuracy = (y_test_pred == y_test).mean() * 100
        test_acc_dist.append(accuracy)

    plt.hist(test_acc_dist)
    plt.title("Histogram of prediction accuracies on test dataset")
    plt.xlabel("Accuracy")
    plt.ylabel("Frequency")
    save_dir = os.path.join(DATA_DIR, "..", "Plots")
    plt.savefig(os.path.join(save_dir,  "Test_Dataset_Prediction_Accuracy.png"))

    # Get the average accuracy
    Y_pred = np.argmax(mean_probs, axis = 1)
    print("Overall Accuracy in predicting the test data =  percent", round((Y_pred == y_test).mean() * 100,2))

In [ ]:
 '''
    # Draw two random samples from the test data
    sample_images_idx= np.random.choice(range(X_test.shape[0]), size=10)
    for i in sample_images_idx:
        sampled_image = X_test[i]
        sample_label = y_test[i]
        mean_prediction = Y_pred[i]
        plot_heldout_prediction(sampled_image, probs[:,i,:],
                                fname="Sample{:05d}_pred".format(i),
                                title="Correct Label {:02d}, Mean Prediction {:02d}"
                                .format(sample_label,mean_prediction))

    qm_vals, qs_vals = sess.run((qmeans, qstds))

    # Plotting Weight Means and Standard deviation
    plot_weight_posteriors(names, qm_vals, qs_vals,
                           fname="step{:05d}_weights.png".format(step))
'''